In [4]:
import numpy as np
import pmdarima as pm
from pmdarima import model_selection
from pmdarima.model_selection import SlidingWindowForecastCV
from pmdarima.model_selection import cross_val_score

ModuleNotFoundError: No module named 'pmdarima'

In [21]:
data = pm.datasets.load_wineind()
tr, te = train_test_split(data, train_size=165)

model1 = pm.ARIMA(order=(2, 1, 1))
model2 = pm.ARIMA(order=(1, 1, 2), seasonal_order=(0, 1, 1, 12), suppress_warnings=True)

cv = SlidingWindowForecastCV(h=1 step=24, window_size=100)

scores_model1 = cross_val_score(estimator=model1, y=tr, scoring="smape", cv=cv, verbose=2)
scores_model2 = cross_val_score(estimator=model2, y=tr, scoring="smape", cv=cv, verbose=2)

print(f"scores_model1: {scores_model1}")
print(f"scores_model2: {scores_model2}")

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
scores_model1: [23.92897667 22.28966615  3.74840893]
scores_model2: [ 1.52039644 15.41342726  5.57222576]
